# Block 1 (Filtering of our Data)

In [ ]:
import glob
import pandas as pd

def filter_data_by_date_and_index(file_pattern, input_date, segment, index):
    # Step 1: Loading all relevant CSV files using the glob function
    csv_files = glob.glob(file_pattern)
    dataframes = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            # Renaming "Date" to "date" and converting to the correct format
            df.rename(columns={"Date": "date"}, inplace=True)
            df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
            dataframes.append(df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    # Step 2: Combining all data into one DataFrame using the concat method
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Step 3: Filtering data by date
    filtered_by_date = combined_df[combined_df['date'] == input_date]

    # Step 4: Filtering by Ticker - applying segment filter first, then index
    filtered_by_ticker = filtered_by_date[
        filtered_by_date['Ticker'].str.contains(segment, case=False) & 
        filtered_by_date['Ticker'].str.contains(index, case=False)]
    
    return filtered_by_ticker

# File pattern and inputs
file_pattern = "*_BACKADJUSTED_*.csv"  
input_date = input("Enter a date (YYYY-MM-DD): ")
segment = input("Enter the segment (e.g., BFO/NFO/MCX): ")
index = input("Enter the index (e.g., CRUDEOIL, GOLD): ")

# Execute function and print the result
data = filter_data_by_date_and_index(file_pattern, input_date, segment, index)
print(data)


# Block 2 (Leg Execution)

In [1]:
# Function to get ITM/OTM level dynamically
def get_strike_level(strike_selection):
    import re
    if strike_selection == 'ATM':
        return 0, 'ATM'
    match = re.match(r'(ITM|OTM)-(\d+)', strike_selection)
    return int(level), strike_type


segment_data = pd.DataFrame({
    'segment': ['NIFTY', 'BANKNIFTY', 'BANKEX', 'SENSEX', 'CRUDEOIL', 'SILVER', 'GOLD', 'NICKEL', 'ZINC'],
    'lot_size': [25, 15, 15, 10, 100, 30, 100, 1500, 5000],
    'strike_diff': [50, 100, 100, 100, 50, 50, 50, 50, 50]
})

def get_all_expiry_dates(data, input_date, index, option_type, segment, selected_strike):
    
    # Parsing the input date to extract the month and year
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    month_abbr = input_date_obj.strftime("%b").upper()  
    
    # Constructing the ticker pattern
    ticker_pattern = f"{index}d{{2}}{month_abbr}{selected_strike}{option_type}.{segment}"

    # Filtering tickers matching the pattern
    filtered_tickers = data[data['Ticker'].str.contains(ticker_pattern, regex=True)]

    # Extracting unique days from the filtered tickers
    expiry_days = set()
    for ticker in filtered_tickers['Ticker']:
        # Extracting the day (2 digits before the month abbreviation)
        day_str = ticker.split(index)[1][:2]
        try:
            expiry_days.add(int(day_str))  # Ensure it's a valid integer
        except ValueError:
            continue

    # Combining days with the input month and year
    expiry_dates = [datetime(year, input_date_obj.month, day) for day in sorted(expiry_days)]

    return expiry_dates


# also include to take input of "what kind of expiry i want" and "DTE" {Days to expiry}
# "what kind of expiry i want" means daily/weekly/monthly
def leg_function(data, index, action, option_type, lots, strike_selection, 
                entry_time, universal_exit_time, stop_loss, stop_loss_type, target, target_type, 
                segment_data):
    """
    Function to determine trade details for options based on market data and parameters.
    """
    import pandas as pd
    
    # Getting strike difference for the "segment_data"
    strike_diff = segment_data.loc[segment_data['segment'] == index, 'strike_diff'].iloc[0]
    
    # Constructing the ticker name from index and Segment
    ticker_name = f"{index}-I.{Segment}"

    # Get the entry spot price by finding the close price at the entry time for the given ticker
    entry_data = data[(data['symbol'] == ticker_name) & (data['time'] == entry_time)]
    entry_spot = float(entry_data['close'].iloc[0])
    
    # Calculating ATM strike (round spot price to nearest strike difference multiple)
    atm_strike = round(entry_spot / strike_diff) * strike_diff
    
    # Calculate the selected strike by use of "get_strike_level()" function
    level, strike_type = get_strike_level(strike_selection)
    if strike_type == 'ITM':
        multiplier = -1 if option_type == 'CE' else 1
    elif strike_type == 'OTM':
        multiplier = 1 if option_type == 'CE' else -1
    selected_strike = atm_strike + (multiplier * level * strike_diff)
    
    # Getting all possible expiry dates in an array
    expiry_dates = get_all_expiry_dates(data, input_date, index, option_type, segment, selected_strike)
    
    # From all element in "get_all_expiry_dates" we will get the required date and get "spot"
    # see if expiry is weekly , monthly or daily
    
    # Calculating stop loss and target levels
    stop_loss_level = spot * stop_loss_type if stop_loss == 'yes' else None
    target_level = spot * target_type if target == 'yes' else None
    
    # Filtering data for monitoring (from entry time onwards)
    # we will observe that "date"{from input_date} and "Ticker"{correct} and we will get "monitoring_data"

    monitoring_data = data[
        (data['symbol'] == index) & 
        (data['time'] >= entry_time) & 
        (data['time'] <= universal_exit_time)
    ].sort_values('time')
    
    # Initializing exit variables
    exit_spot = None
    exit_reason = None
    actual_exit_time = None
    
    # Monitoring price movement to find exit point
    for idx, row in monitoring_data.iterrows():
        current_spot = float(row['close'])
        current_time = row['time']
        
        # Checking stop loss
        if stop_loss == 'yes':
            if ((action == 'BUY' and current_spot <= stop_loss_level) or 
                (action == 'SELL' and current_spot >= stop_loss_level)):
                exit_spot = current_spot
                exit_reason = 'Stop Loss'
                actual_exit_time = current_time
                break
        
        # Checking target
        if target == 'yes':
            if ((action == 'BUY' and current_spot >= target_level) or 
                (action == 'SELL' and current_spot <= target_level)):
                exit_spot = current_spot
                exit_reason = 'Target'
                actual_exit_time = current_time
                break
        
        # Checking universal exit time
        if current_time == universal_exit_time:
            exit_spot = current_spot
            exit_reason = 'Time Exit'
            actual_exit_time = current_time
            break
    
    # If no exit condition was met before universal_exit_time, use the last available price
    if exit_spot is None:
        last_data = monitoring_data.iloc[-1]
        exit_spot = float(last_data['close'])
        exit_reason = 'Time Exit'
        actual_exit_time = last_data['time']
    
    # Calculate PnL
    pnl = (exit_spot - entry_spot) * lots if action == 'BUY' else -(exit_spot - entry_spot) * lots
    
    # Return all calculated values
    trade_details = {
        'entry_time': entry_time,
        'entry_spot': entry_spot,
        'atm_strike': atm_strike,
        'selected_strike': selected_strike,
        'option_type': option_type,
        'action': action,
        'lots': lots,
        'stop_loss_level': stop_loss_level,
        'target_level': target_level,
        'exit_time': actual_exit_time,
        'exit_spot': exit_spot,
        'exit_reason': exit_reason,
        'universal_exit_time': universal_exit_time,
        'pnl': pnl
    }
    
    return trade_details

# Leg 1
leg1 = leg_function(
    data=data,
    Index='GOLD',
    action='BUY',
    option_type='CE',
    lots=1,
    strike_selection='ITM-1',
    entry_time='09:00',
    universal_exit_time='10:15',
    stop_loss='yes',
    stop_loss_type=0.97,
    target='yes',
    target_type=1.05,
    segment_data=segment_data
)

print(f"Leg 1: {leg1}")

# Leg 2 (entry time is the exit time of leg 1 if within universal_exit_time)
if leg1['exit_time'] < '10:15':  # Universal exit time
    leg2 = leg_function(
        data=data,
        Index='GOLD',
        action='SELL',
        option_type='PE',
        lots=2,
        strike_selection='OTM-1',
        entry_time=leg1['exit_time'],  # Exit time of leg 1
        universal_exit_time='10:15',
        stop_loss='yes',
        stop_loss_type=0.95,
        target='yes',
        target_type=1.04,
        segment_data=segment_data
        # expiry type
        # DTE
    ) 
    print(f"Leg 2: {leg2}")

# Leg 3 (manually chosen entry time)
leg3 = leg_function(
    data=data,
    Index='GOLD',
    action='BUY',
    option_type='CE',
    lots=1,
    strike_selection='ATM',
    entry_time='10:00',
    universal_exit_time='10:15',
    stop_loss='yes',
    stop_loss_type=0.96,
    target='yes',
    target_type=1.06,
    segment_data=segment_data
)

print(f"Leg 3: {leg3}")


SyntaxError: invalid syntax (2054432825.py, line 55)